<a href="https://colab.research.google.com/github/nguyenhaphan1/ID-Recognition-With-Contour-Separation-And-Classifier/blob/main/IDRecognitionCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install tensorflow==2.15.1 keras==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import shutil
from google.colab.patches import cv2_imshow
import warnings
import tensorflow as tf
import keras
warnings.filterwarnings('ignore')

In [5]:
print(tf.__version__)
print(keras.__version__)

2.15.1
2.15.0


In [6]:
classifier = tf.keras.models.load_model('/content/drive/MyDrive/CCCD/digit_classification.keras')

In [25]:
def draw_bounding_box_with_contour(img, contour):
  x, y, w, h = cv2.boundingRect(contour)
  cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

def draw_bounding_box_with_bbox(img, bbox):
  cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 1)
  return img

def finding_narrowest_position(contours):
  narrowest_distance = 999
  narrowest_position = 0

  while(len(contours) > 0):
    x_cnt = []
    idx = 0
    x_value = contours[idx][0]
    # print(f"X value: {x_value}")

    while(idx < len(contours)):
      if contours[idx][0] == x_value:
        x_cnt.append(contours[idx])
        idx += 1
      else:
        break

    x_cnt = sorted(x_cnt, key=lambda x:x[1])
    cnt_distance = x_cnt[-1][1] - x_cnt[0][1]

    if cnt_distance < narrowest_distance:
      narrowest_distance = cnt_distance
      narrowest_position = x_value

    contours = contours[idx:]

  return narrowest_position


def digits_recognition(bboxes, model, target_size, img):

  digits = ""
  for bbox in bboxes:
    x, y, w, h = bbox
    digit_img = img[y:y+h, x:x+w]
    digit_img = cv2.resize(digit_img, (target_size, target_size))
    digit_img = digit_img.reshape(1, target_size, target_size, 3)
    prob = model.predict(digit_img, verbose=False)
    digits += str(np.argmax(prob))

  return digits

In [26]:
import os
import cv2
import numpy as np

from google.colab.patches import cv2_imshow

# Threshold ratios and constants
min_ratios = {
    "single": 0.036231884057971016,
    "two": 0.13559322033898305,
    "three": 0.21008403361344538,
    "four": 0.29411764705882354,
    "five": 0.3813559322033898,
    "six": 0.4491525423728814,
    "seven": 0.5289256198347108,
    "eight": 0.625,
    "nine": 0.7,
    "ten": 0.7844827586206896
}
min_area = 999
min_ratio_height = 0.5428571428571428  # Adjusted initial height threshold ratio

# Function to draw contours and process images
def process_image(image_path):
    global correct_separation, incorrect_separation, min_ratio_height, saved_file

    img = cv2.imread(image_path)
    if img is None:
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image_width, image_height = img_gray.shape[1], img_gray.shape[0]
    image_area = image_height * image_width

    thresh1 = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 93, 19)
    thresh1 = cv2.bitwise_not(thresh1)

    contours, _ = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    bboxes = [cv2.boundingRect(contour) for contour in contours if cv2.moments(contour)["m00"] != 0]
    bboxes.sort(key=lambda x: x[0])

    # Flatten contours for further calculations
    flatten_contours = np.concatenate(contours, axis=0).reshape((-1, 2))

    # Detect digits in bounding boxes
    true_bboxes = detect_digits(bboxes, flatten_contours, image_width, image_height, img_gray)

    # Evaluate detection
    if evaluate_detection(true_bboxes, image_height):
      result = digits_recognition(true_bboxes, classifier, target_size=32, img=img_rgb)
      print(f"Result: {result}")
      print()

# Function to detect digits within bounding boxes
def detect_digits(bboxes, flatten_contours, image_width, image_height, img_gray):
    true_bboxes = []

    for bbox in bboxes:
        ratio_width = bbox[2] / image_width
        ratio_height = bbox[3] / image_height
        x, y, w, h = bbox
        if ratio_height >= min_ratio_height:
            if min_ratios["single"] <= ratio_width < min_ratios["two"]:
                true_bboxes.append(bbox)
                min_ratios["single"] = min(min_ratios["single"], ratio_width)
            elif min_ratios["two"] <= ratio_width < min_ratios["three"]:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [0.25, 0.75]))
                min_ratios["two"] = min(min_ratios["two"], ratio_width)
            elif min_ratios["three"] <= ratio_width < min_ratios["four"]:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [1/6, 1/2, 5/6], separate_into=3))
                min_ratios["three"] = min(min_ratios["three"], ratio_width)
            elif min_ratios["four"] <= ratio_width < min_ratios["five"]:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [1/8, 3/8, 5/8, 7/8], separate_into=4))
                min_ratios["four"] = min(min_ratios["four"], ratio_width)
            elif min_ratios["five"] <= ratio_width < min_ratios["six"]:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [1/10, 3/10, 1/2, 7/10, 9/10], separate_into=5))
            elif min_ratios["six"] <= ratio_width < min_ratios["seven"]:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [1/12, 3/12, 5/12, 7/12, 9/12, 11/12], separate_into=6))
            elif min_ratios["seven"] <= ratio_width < min_ratios["eight"]:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [1/14, 3/14, 5/14, 7/14, 9/14, 11/14, 13/14], separate_into=7))
            elif min_ratios["eight"] <= ratio_width < min_ratios["nine"]:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [1/16, 3/16, 5/16, 7/16, 9/16, 11/16, 13/16, 15/16], separate_into=8))
            elif min_ratios["nine"] <= ratio_width < min_ratios["ten"]:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [1/18, 3/18, 5/18, 7/18, 9/18, 11/18, 13/18, 15/18, 17/18], separate_into=9))
            elif min_ratios["ten"] <= ratio_width < 0.8:
                true_bboxes.extend(separate_bbox(bbox, flatten_contours, [1/20, 3/20, 5/20, 7/20, 9/20, 11/20, 13/20, 15/20, 17/20, 19/20], separate_into=10))
        else:
            print("Not match height of a digit")

    return true_bboxes

# Function to separate bounding boxes
def separate_bbox(bbox, flatten_contours, positions, separate_into=2):
    result_bboxes = []
    contours = [sorted([[x, y] for x, y in flatten_contours if bbox[0] + pos[0]*bbox[2] < x < bbox[0] + pos[1]*bbox[2]], key=lambda x: x[0]) for pos in zip(positions[:-1], positions[1:])]

    x_divides = [finding_narrowest_position(contour) for contour in contours]
    x_divides = [bbox[0]] + x_divides + [bbox[0] + bbox[2]]

    for i in range(separate_into):
        result_bboxes.append((x_divides[i], bbox[1], x_divides[i+1] - x_divides[i], bbox[3]))

    return result_bboxes

# Function to evaluate the detection results
def evaluate_detection(true_bboxes, image_height):
    global correct_separation, incorrect_separation, min_ratio_height

    if len(true_bboxes) == 12:
        min_ratio_height = min(min_ratio_height, min([bbox[3] / image_height for bbox in true_bboxes]))
        return True
    else:
        print(f"Wrong separation: Length: {len(true_bboxes)}")
        return False


image_path = '/content/drive/MyDrive/CCCD/data_for_prediction/new_ids_separation2/id_15.jpg'
process_image(image_path)

Result: 030093009624

